In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset # 더 이상 사용하지 않음
import soundfile as sf
import numpy as np # <--- [수정 1] numpy 임포트 추가
import tqdm # tqdm 모듈 임포트

# --- (이전 코드는 동일하게 유지) ---
# 1. 장치 설정
if torch.backends.mps.is_available():
    device = "mps"
    torch_dtype = torch.float16
elif torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32

print(f"🚀 사용 장치: {device}")
print(f"💡 사용 데이터 타입: {torch_dtype}")

model_id = "openai/whisper-large-v3-turbo" # 또는 사용 중인 모델 ID

# 2. 모델 로드
print(f"모델 ({model_id}) 로딩 중... (장치: {device}, 데이터 타입: {torch_dtype})")
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True if device == "cpu" else False,
    use_safetensors=True
)
model.to(device)
print("✅ 모델 로딩 완료")

# 3. 프로세서 로드
processor = AutoProcessor.from_pretrained(model_id)
print("✅ 프로세서 로딩 완료")

# 4. 파이프라인 생성
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)
print("✅ 음성 인식 파이프라인 생성 완료")

🚀 사용 장치: mps
💡 사용 데이터 타입: torch.float16
모델 (openai/whisper-large-v3-turbo) 로딩 중... (장치: mps, 데이터 타입: torch.float16)
✅ 모델 로딩 완료


Device set to use mps


✅ 프로세서 로딩 완료
✅ 음성 인식 파이프라인 생성 완료


In [8]:
# your_audio_file_path = "/Users/gim-yonghyeon/Documents/GitHub/stt_project/광화문자생한방병원_1.mp3"
your_audio_file_path = "/Users/gim-yonghyeon/Downloads/멕시코 스페인어.mp3"

try:
    print(f"로컬 오디오 파일 로딩 중: {your_audio_file_path}")
    if your_audio_file_path == "YOUR_AUDIO_FILE.wav":
        raise FileNotFoundError("오디오 파일 경로를 실제 파일 경로로 수정해주세요. 'YOUR_AUDIO_FILE.wav'는 예시입니다.")
    audio_array, sampling_rate = sf.read(your_audio_file_path)
    if audio_array.dtype != np.float32:
        print(f"오디오 배열을 float32로 변환 중 (현재 dtype: {audio_array.dtype})")
        audio_array = audio_array.astype(np.float32)
    if audio_array.ndim > 1 and audio_array.shape[1] > 1:
        print(f"스테레오 오디오(채널 수: {audio_array.shape[1]})를 모노로 변환 중...")
        audio_array = np.mean(audio_array, axis=1)
    loaded_audio_sample = {
        "path": your_audio_file_path,
        "array": audio_array,
        "sampling_rate": sampling_rate
    }
    print(f"✅ 로컬 오디오 파일 로딩 완료: {loaded_audio_sample['path']}")
    print(f"   오디오 길이 (샘플 수): {len(loaded_audio_sample['array'])}")
    print(f"   샘플링 레이트: {loaded_audio_sample['sampling_rate']} Hz")
    print(f"   오디오 배열 dtype: {loaded_audio_sample['array'].dtype}")

except FileNotFoundError as e:
    print(f"❌ 파일 오류: {e}")
    print("지정한 경로에 오디오 파일이 있는지 확인해주세요.")
    exit()
except Exception as e:
    print(f"❌ 오디오 파일 로드 중 오류 발생: {e}")
    print("지원되는 오디오 파일 형식(예: WAV, MP3, FLAC 등)인지 확인해주세요.")
    print("다양한 오디오 형식을 지원하려면 시스템에 FFmpeg가 설치되어 있어야 합니다.")
    exit()

# --- [수정된 부분 시작] ---

# sdpa_kernel과 SDPBackend를 사용하려면 상단에 import 구문이 필요합니다.
# 예: from torch.nn.attention import sdpa_kernel, SDPBackend
# 만약 이 기능이 확실하지 않다면, 아래 'with sdpa_kernel(...)' 줄들을 제거하고
# 'pipe' 함수 호출만 남겨도 기본적인 동작에는 문제가 없습니다.

print("\n🔥 워밍업 단계 시작 (모델 최적화를 위해 몇 차례 실행)...")
try:
    if 'loaded_audio_sample' in locals(): # 오디오 파일이 성공적으로 로드되었는지 확인
        # tqdm 사용법 수정 및 올바른 변수명 사용
        for _ in tqdm.tqdm(range(2), desc="Warm-up step"): # <--- [수정 2] tqdm.tqdm()으로 변경
            # loaded_audio_sample 사용, 불필요할 수 있는 generate_kwargs 제거 (단순 워밍업)
            _ = pipe(loaded_audio_sample.copy()) # <--- [수정 3] sample -> loaded_audio_sample
        print("✅ 워밍업 완료")

        print("\n🚀 최종 음성 인식 실행...")
        # 여기도 loaded_audio_sample 사용
        result = pipe(loaded_audio_sample.copy(), generate_kwargs={"language": None}) # <--- [수정 3] sample -> loaded_audio_sample

        print("\n📄 최종 변환된 텍스트:")
        print(result)
    else:
        print("❌ 오디오 데이터가 로드되지 않아 음성 인식을 진행할 수 없습니다.")

except NameError as e:
    # sdpa_kernel 등이 import되지 않았을 경우 발생 가능
    print(f"❌ 실행 중 변수 이름 관련 오류 발생: {e}")
    print("   'sdpa_kernel' 또는 'SDPBackend' 같은 특별한 기능을 사용하려 했다면, 해당 기능이 올바르게 import 되었는지 확인해주세요.")
    print("   또는 해당 코드 라인을 일시적으로 주석 처리하거나 제거하고 다시 시도해보세요.")
except Exception as e:
    print(f"❌ 음성 인식 실행 중 예상치 못한 오류 발생: {e}")

# --- [수정된 부분 끝] ---

로컬 오디오 파일 로딩 중: /Users/gim-yonghyeon/Downloads/멕시코 스페인어.mp3
오디오 배열을 float32로 변환 중 (현재 dtype: float64)
스테레오 오디오(채널 수: 2)를 모노로 변환 중...
✅ 로컬 오디오 파일 로딩 완료: /Users/gim-yonghyeon/Downloads/멕시코 스페인어.mp3
   오디오 길이 (샘플 수): 783360
   샘플링 레이트: 44100 Hz
   오디오 배열 dtype: float32

🔥 워밍업 단계 시작 (모델 최적화를 위해 몇 차례 실행)...


Warm-up step: 100%|███████████████████████████████| 2/2 [00:06<00:00,  3.07s/it]


✅ 워밍업 완료

🚀 최종 음성 인식 실행...

📄 최종 변환된 텍스트:
{'text': ' Empezando la grabación de voz. Suspendiendo la grabación de voz. Activando el modo de aparcamiento. El modo de aparcamiento se ha desactivado. Formateando la tarjeta SD. Inalizando la tarjeta SD. Cambiar la tarjeta SD.', 'chunks': [{'timestamp': (0.0, 1.88), 'text': ' Empezando la grabación de voz.'}, {'timestamp': (2.46, 4.24), 'text': ' Suspendiendo la grabación de voz.'}, {'timestamp': (4.88, 6.8), 'text': ' Activando el modo de aparcamiento.'}, {'timestamp': (7.32, 9.9), 'text': ' El modo de aparcamiento se ha desactivado.'}, {'timestamp': (10.38, 12.18), 'text': ' Formateando la tarjeta SD.'}, {'timestamp': (12.88, 14.86), 'text': ' Inalizando la tarjeta SD.'}, {'timestamp': (15.68, 17.46), 'text': ' Cambiar la tarjeta SD.'}]}


In [9]:
result

{'text': ' Empezando la grabación de voz. Suspendiendo la grabación de voz. Activando el modo de aparcamiento. El modo de aparcamiento se ha desactivado. Formateando la tarjeta SD. Inalizando la tarjeta SD. Cambiar la tarjeta SD.',
 'chunks': [{'timestamp': (0.0, 1.88),
   'text': ' Empezando la grabación de voz.'},
  {'timestamp': (2.46, 4.24), 'text': ' Suspendiendo la grabación de voz.'},
  {'timestamp': (4.88, 6.8), 'text': ' Activando el modo de aparcamiento.'},
  {'timestamp': (7.32, 9.9),
   'text': ' El modo de aparcamiento se ha desactivado.'},
  {'timestamp': (10.38, 12.18), 'text': ' Formateando la tarjeta SD.'},
  {'timestamp': (12.88, 14.86), 'text': ' Inalizando la tarjeta SD.'},
  {'timestamp': (15.68, 17.46), 'text': ' Cambiar la tarjeta SD.'}]}

In [2]:
from transformers import pipeline
import soundfile as sf
import numpy as np
import torch # torch 가용 여부에 따라 device 설정

stt_model_name = "openai/whisper-large-v3-turbo" # 또는 "openai/whisper-large-v3" 사용 권장
audio_file_path = "/Users/gim-yonghyeon/Downloads/광화문자생한방병원_1.mp3" # 실제 파일 경로로 변경

if torch.backends.mps.is_available():
    device = "mps"
    print("MPS 장치를 사용합니다.")
elif torch.cuda.is_available():
    device = "cuda"
    print("CUDA 장치를 사용합니다.")
else:
    device = "cpu"
    print("CPU를 사용합니다.")

try:
    audio_input, sample_rate = sf.read(audio_file_path)
    print(f"오디오 파일 로드 완료: {audio_file_path}, 샘플링 레이트: {sample_rate}")

    if audio_input.ndim > 1: 
        print(f"오디오를 모노로 변환 중 (채널 수: {audio_input.shape[1]})")
        audio_input = np.mean(audio_input, axis=1)
    
    if audio_input.dtype != np.float32:
        print(f"오디오 타입을 np.float32로 변환 중 (현재 타입: {audio_input.dtype})")
        audio_input = audio_input.astype(np.float32)
        
    # 1. Whisper 파이프라인 생성 (워밍업 전에 생성!)
    # 1. Whisper 파이프라인 생성 (워밍업 전에 생성!)
    print(f"Whisper 파이프라인 생성 중 (모델: {stt_model_name}, 장치: {device})...")
    whisper_pipeline = pipeline(
        "automatic-speech-recognition",
        model=stt_model_name,
        torch_dtype=torch.float16 if device == "mps" else torch.float32,
        device=device,
        return_timestamps=True  # <--- 여기에도 추가! (스크립트 1과 동일하게)
    )
    print("Whisper 파이프라인 생성 완료.")

    # 2. 워밍업 단계 시작
    print("\n🔥 워밍업 단계 시작 (모델 최적화를 위해 몇 차례 실행)...")
    for i in range(2):
        print(f"Warm-up run #{i+1}...")
        # return_timestamps는 파이프라인 생성시 설정했으므로 호출시엔 생략 가능 (또는 일관성 위해 True 전달도 무방)
        _ = whisper_pipeline(audio_input.copy()) 
    print("✅ 워밍업 완료")

    # 3. 실제 음성 인식 작업 실행 (한국어 전사)
    print("\n🚀 오디오 -> 한국어 전사 작업 실행 중...")
    result = whisper_pipeline(
        audio_input.copy()
    )
    transcribed_text = result['text']
    print(f"Whisper output (Korean Transcription): {transcribed_text}")

except FileNotFoundError:
    print(f"오류: 오디오 파일을 찾을 수 없습니다 - {audio_file_path}")
except Exception as e:
    print(f"스크립트 실행 중 오류 발생: {e}")

MPS 장치를 사용합니다.
오디오 파일 로드 완료: /Users/gim-yonghyeon/Downloads/광화문자생한방병원_1.mp3, 샘플링 레이트: 44100
오디오 타입을 np.float32로 변환 중 (현재 타입: float64)
Whisper 파이프라인 생성 중 (모델: openai/whisper-large-v3-turbo, 장치: mps)...


Device set to use mps


Whisper 파이프라인 생성 완료.

🔥 워밍업 단계 시작 (모델 최적화를 위해 몇 차례 실행)...
Warm-up run #1...
Warm-up run #2...
✅ 워밍업 완료

🚀 오디오 -> 한국어 전사 작업 실행 중...
Whisper output (Korean Transcription):  책주 디스크 가슴수출을 영양은 자생환동병연으로 가셨다면 소감영역 6번 출고를 나가시길 바랍니다.
